In [1]:
!pip install -U catalyst
!pip install scikit-learn

     |████████████████████████████████| 491kB 7.7MB/s 
     |████████████████████████████████| 317kB 14.3MB/s 


In [1]:
from torch.utils.data import Dataset, DataLoader
import torch.nn.functional as F
import torch.nn as nn
import torch.optim as optim
import torch

import torchvision.transforms as T
import torchvision.transforms.functional as VF

import numpy as np
from numpy.random import default_rng
import os
from os import makedirs, path
from tqdm.notebook import tqdm
from os.path import join

from sklearn.model_selection import train_test_split

from PIL import Image, ImageDraw

from catalyst.dl import Runner, SupervisedRunner
from catalyst.callbacks.scheduler import SchedulerCallback
from catalyst.callbacks.early_stop import EarlyStoppingCallback
from catalyst.utils import set_global_seed, prepare_cudnn

import matplotlib.pyplot as plt

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

torch.manual_seed(42)

In [41]:


orig_width, orig_height = 640, 640
width, height = 640, 640

func = lambda x: x ** 4
max_x = np.pi / 2

linewidth = 4


def img_threshold(img, thresh=150):
    fn = lambda x : 255 if x > thresh else 0
    img = img.convert('L').point(fn, mode='1')
    return img


def to(values, out_min, out_max):
    in_min = values.min()
    in_max = values.max()
    return (values - in_min) * (out_max - out_min) / (in_max - in_min) + out_min


def generate_points(func, abs_value=np.pi, size=1000):
    x = np.linspace(-1, 1, size)
    y = to(func(x * abs_value), -1, 1)
    return x, y


def randomize(x, y, eps=1e-2, seed=42, random_state=None):
    if random_state is None:
        random_state = default_rng(seed)
    x = (x + random_state.uniform(-eps, eps, x.shape[0]))
    y = (y + random_state.uniform(-eps, eps, y.shape[0])) * random_state.choice([1, -1])
    return x, y


def bound_points(x, y, seed=42, random_state=None):
    if random_state is None:
        random_state = default_rng(seed)
    bound = random_state.uniform(0.3, 1.0, 2)
    x *= bound[0]
    y *= bound[1]
    return x, y


def offset_points(x, y, seed=42, random_state=None):
    if random_state is None:
        random_state = default_rng(seed)
    bound = random_state.uniform(0.3, 1.0, 2)
    offset = random_state.uniform((-1 + bound) / 2, (1 - bound) / 2)
    x += offset[0]
    y += offset[1]
    return x, y


def full_randomize(x, y, seed=42, do_randomize=True):
    if do_randomize:
        x, y = randomize(x, y, seed=seed)
    x, y = bound_points(x, y, seed=seed)
    x, y = offset_points(x, y, seed=seed)
    return x, y


def create_image(x, y, 
                 orig_size=(orig_width, orig_height),
                 size=(width, height),
                 linewidth=4, do_threshold=True):
    img = Image.new('RGB', orig_size, color='black')

    draw = ImageDraw.Draw(img)

    x = x * orig_size[0] / 2 + orig_size[0] / 2
    y = y * orig_size[1] / 2 + orig_size[1] / 2

    for xi, yi in zip(x, y):
        draw.ellipse([(xi - linewidth / 2, yi - linewidth / 2),
                      (xi + linewidth / 2, yi + linewidth / 2)],
                     fill='white')
    
    if do_threshold:
        img = img_threshold(img)
    img = img.resize(size)

    return img

seed = 42
x, y = full_randomize(*generate_points(func, abs_value=max_x), seed=seed, do_randomize=False)
create_image(x, y, linewidth=linewidth).save('test.png')

x, y = generate_points(func, abs_value=max_x)
x_rand, y_rand = full_randomize(x, y, seed=seed)
create_image(x_rand, y_rand, linewidth=linewidth).save('test2.png')

In [3]:
width, height = 64, 64
class DocDataset(Dataset):

    def __init__(self, functions, num_points=1000, num_anchor_points=100, 
                 width=width, height=height, size=1000, train=True):
        super().__init__()
        
        self.functions = functions
        self.num_points = num_points
        self.step = self.num_points // num_anchor_points
        self.width = width
        self.height = height
        self.size = size
        self.train = train

        self.randomize_func = randomize if self.train else lambda x, y, **kwargs: (x, y)

        self.transform = T.Compose([
                                    T.ToTensor(),
                                    ])
        
        self.imgs = []
        self.points = []
        for i in tqdm(range(size)):
            start_points = generate_points(self.functions[i % len(self.functions)],
                                           np.pi/2, self.num_points)
            start_points = self.randomize_func(*start_points, seed=i)
            start_points = full_randomize(*start_points, seed=i, do_randomize=False)
            x = create_image(*start_points, size=(self.width, self.height))
            x = self.transform(x)
            start_points = np.vstack(start_points)
            start_points = np.moveaxis(start_points.astype(np.float32), 0, -1)
            self.imgs.append(x)
            self.points.append(start_points[:-1:self.step, :])
    
    def __len__(self):
        return self.size
    
    def __getitem__(self, i):
        # start_points = generate_points(self.functions[i % len(self.functions)],
        #                                np.pi/2, self.num_points)
        # start_points = np.vstack(start_points)
        # points = self.randomize_func(*start_points, seed=i)
        # points = bound_points(*points, seed=i)
        # x = create_image(*points, size=(self.width, self.height))
        # x = self.transform(x)
        # start_points = np.moveaxis(start_points.astype(np.float32), 0, -1)
        # return {'x': x, 'targets': start_points[:-1:self.step, :]}
        return {'x': self.imgs[i], 'targets': self.points[i]}

In [4]:
class Model(nn.Module):
    def __init__(self, conv_drop=0.2, hid_size=128, layers=4, gru_drop=0.3, lin_drop=0.5):
        super().__init__()
        self.block1 = nn.Sequential(nn.Conv2d(1, 16, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(16),
                                     nn.LeakyReLU(),
                                     nn.AdaptiveMaxPool2d((32, 32)))

        self.block2 = nn.Sequential(nn.Conv2d(16, 24, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(24),
                                     nn.LeakyReLU(),
                                     nn.AdaptiveMaxPool2d((24, 24)))
        
        self.block3 = nn.Sequential(nn.Conv2d(24, 32, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(32),
                                     nn.LeakyReLU(),
                                     nn.AdaptiveMaxPool2d((16, 16)))
        
        self.block4 = nn.Sequential(nn.Dropout2d(conv_drop),
                                     nn.Conv2d(32, 48, kernel_size=3, padding=1),
                                     nn.BatchNorm2d(48),
                                     nn.LeakyReLU(),
                                     nn.AdaptiveMaxPool2d((8, 8)))
        
        self.flat = nn.Flatten(2)
        self.dropout1 = nn.Dropout(lin_drop)
        self.linear1 = nn.Linear(48, 100)
        self.gru = nn.GRU(64, hid_size, num_layers=layers, batch_first=True, 
                          bidirectional=True, dropout=gru_drop)
        self.dropout2 = nn.Dropout(lin_drop)
        self.linear2 = nn.Linear(hid_size * 2, 2)
        self.tanh = nn.Tanh()

    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)

        x = self.flat(x)
        x = x.transpose(1, 2)
        x = self.linear1(self.dropout1(x))
        x = x.transpose(1, 2)
        x, _ = self.gru(x)
        x = self.linear2(self.dropout2(x))
        x = self.tanh(x)

        return x

model = Model()

In [5]:
functions = [np.sin, np.sinh, np.exp, lambda x: x ** 2, lambda x: x ** 3, np.exp2, lambda x: x ** 4]

train_dataset = DocDataset(functions, size=6000)
valid_dataset = DocDataset(functions, size=600, train=False)

In [6]:
batch_size = 32

train_dataloader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True, drop_last=False)
valid_dataloader = DataLoader(valid_dataset, batch_size=batch_size)

dataloaders = {
    "train": train_dataloader,
    "valid": valid_dataloader   
}

In [7]:
seed = 404
set_global_seed(seed)
prepare_cudnn(True)

In [8]:
epochs = 200
lr = 1e-3
criterion = nn.MSELoss(reduction='sum')
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()), lr=lr)
scheduler = optim.lr_scheduler.ReduceLROnPlateau(optimizer, factor=0.7,
                                                 patience=20, min_lr=0.00001)
# scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=20, verbose=True)

In [9]:
model.train()
model = model.cuda()

runner = SupervisedRunner(input_key='x',
                          output_key='logits',
                          input_target_key='targets')

runner.train(
    model=model,
    criterion=criterion,
    optimizer=optimizer,
    scheduler=scheduler,
    loaders=dataloaders,
    callbacks=[
        EarlyStoppingCallback(
            patience=40,
            metric="loss",
            minimize=True,
        ),
        SchedulerCallback(reduced_metric="loss"),
    ],
    num_epochs=epochs,
    verbose=True,
    load_best_on_end=True,
    initial_seed=42,
    logdir="/content/drive/My Drive/robotics/logs"
)

1/200 * Epoch (train): 100% 188/188 [00:10<00:00, 18.78it/s, loss=23.193]
1/200 * Epoch (valid): 100% 19/19 [00:00<00:00, 50.52it/s, loss=33.868]
[2021-01-10 13:21:16,548] 
1/200 * Epoch 1 (_base): lr=0.0010 | momentum=0.9000
1/200 * Epoch 1 (train): loss=163.1489
1/200 * Epoch 1 (valid): loss=52.3244
2/200 * Epoch (train): 100% 188/188 [00:09<00:00, 19.28it/s, loss=13.223]
2/200 * Epoch (valid): 100% 19/19 [00:00<00:00, 53.58it/s, loss=25.570]
[2021-01-10 13:21:26,754] 
2/200 * Epoch 2 (_base): lr=0.0010 | momentum=0.9000
2/200 * Epoch 2 (train): loss=42.1824
2/200 * Epoch 2 (valid): loss=36.3108
3/200 * Epoch (train): 100% 188/188 [00:10<00:00, 18.71it/s, loss=11.249]
3/200 * Epoch (valid): 100% 19/19 [00:00<00:00, 52.62it/s, loss=16.758]
[2021-01-10 13:21:37,265] 
3/200 * Epoch 3 (_base): lr=0.0010 | momentum=0.9000
3/200 * Epoch 3 (train): loss=28.2625
3/200 * Epoch 3 (valid): loss=21.0530
4/200 * Epoch (train): 100% 188/188 [00:09<00:00, 19.81it/s, loss=11.032]
4/200 * Epoch (vali

In [10]:
# !mkdir checkpoint
model = model.cpu()
# checkpoint_path = f'/content/drive/My Drive/robotics/{type(model).__name__}.pth'
checkpoint_path = f'{type(model).__name__}.pth'
torch.save(model.state_dict(), checkpoint_path)

In [6]:
checkpoint_path = f'{type(model).__name__}.pth'
model.load_state_dict(torch.load(checkpoint_path))
model.eval()

Model(
  (block1): Sequential(
    (0): Conv2d(1, 16, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(16, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): AdaptiveMaxPool2d(output_size=(32, 32))
  )
  (block2): Sequential(
    (0): Conv2d(16, 24, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(24, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): AdaptiveMaxPool2d(output_size=(24, 24))
  )
  (block3): Sequential(
    (0): Conv2d(24, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): LeakyReLU(negative_slope=0.01)
    (3): AdaptiveMaxPool2d(output_size=(16, 16))
  )
  (block4): Sequential(
    (0): Dropout2d(p=0.2, inplace=False)
    (1): Conv2d(32, 48, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (2): Ba

In [33]:
out_folder = 'results'
makedirs(out_folder, exist_ok=True)

model = model.cpu()
model.eval()


for num in range(len(functions)):
    func = functions[num]
    max_x = np.pi / 2
    linewidth = 4

    x, y = full_randomize(*generate_points(func, abs_value=max_x), seed=10, do_randomize=False)
    img = create_image(x, y, linewidth=linewidth)
    with torch.no_grad():
        tensor = VF.to_tensor(img).unsqueeze(0)
        points = model(tensor)[0]
    img = create_image(x, y, linewidth=linewidth, do_threshold=False)
    draw = ImageDraw.Draw(img)
    for i in range(points.shape[0]):
        pic_x, pic_y = points[i, 0] * orig_width / 2 + orig_width / 2, points[i, 1] * orig_height / 2 + orig_height / 2
        draw.ellipse([(pic_x - linewidth / 2, pic_y - linewidth / 2),
                      (pic_x + linewidth / 2, pic_y + linewidth / 2)],
                     fill='orange')

    img.save(path.join(out_folder, f'out{num}.png'))

!zip -r results results/

updating: results/ (stored 0%)
updating: results/out5.png (deflated 18%)
updating: results/out6.png (deflated 14%)
updating: results/out1.png (deflated 16%)
updating: results/out0.png (deflated 16%)
updating: results/out3.png (deflated 12%)
updating: results/out2.png (deflated 18%)
updating: results/out4.png (deflated 17%)
